In [ ]:
import torch
import torchvision
from torchsummary import summary
from torchvision import transforms
import sys

sys.path.append("/workspaces/ood/")
from ood.train import train_linear_model
import warnings

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
device = torch.device("cuda")
torch.cuda.is_available()

### 1) Load data

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)
cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

svhn_data_train = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform
)
svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

# ViT

### Predict on datasets

In [ ]:
import json
from PIL import Image
import torch
from torchvision import transforms

# Load ViT
from pytorch_pretrained_vit import ViT
model = ViT('B_16_imagenet1k', pretrained=True)
model.eval()

# Load image
# NOTE: Assumes an image `img.jpg` exists in the current directory
transform = transforms.Compose([
    transforms.Resize((384, 384)), 
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])


In [ ]:
from scripts_ood.utils import imagenet_sanity_check

imagenet_sanity_check(model, transform, device)

# Softmax ImageNet resnet50

In [ ]:
soft_model = torchvision.models.resnet50(pretrained=False).to(device)
soft_model.load_state_dict(
    torch.load(
        "/workspaces/ood/data/models/torch/hub/checkpoints/resnet50-0676ba61.pth"
    )
)
summary(soft_model, input_size=(3, 224, 224))

In [ ]:
from ood.utils import imagenet_sanity_check

imagenet_sanity_check(soft_model, transform, device)

In [ ]:
from scripts_ood.utils import add_labels, predict_on_whole_dataset

predict_on_whole_dataset(soft_model, cifar_data_train, "soft_cifar_train")

In [ ]:
model_name = "soft"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=1000)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/soft_cifar_linear_model0.88test_accuracy.pth')

# MoCo v2 ImageNet pretrain

In [ ]:
from ood.utils import load_moco

model = load_moco("/workspaces/ood/data/models/moco_v2_800ep_pretrain.pth.tar", device)

In [ ]:
from ood.utils import add_labels, predict_on_whole_dataset_moco

predict_on_whole_dataset_moco(model, cifar_data_test, "moco_cifar_test", device)
predict_on_whole_dataset_moco(model, cifar_data_train, "moco_cifar_train", device)
predict_on_whole_dataset_moco(model, svhn_data_train, "moco_svhn_train", device)
predict_on_whole_dataset_moco(model, svhn_data_test, "moco_svhn_test", device)

In [ ]:
model_name = "moco"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/moco_cifar_linear_model0.8382test_accuracy.pth')

# BYOL ImageNet pretrain

In [ ]:
from ood.utils import load_byol

model = load_byol("/workspaces/ood/data/models/pretrain_res50x1.pth.tar", device)

model_name = "byol"
predict_on_whole_dataset(model, cifar_data_test, f"{model_name}_cifar_test", device)
predict_on_whole_dataset(model, cifar_data_train, f"{model_name}_cifar_train", device)
predict_on_whole_dataset(model, svhn_data_train, f"{model_name}_svhn_train", device)
predict_on_whole_dataset(model, svhn_data_test, f"{model_name}_svhn_test", device)

In [ ]:
model_name = "byol"

linear_model = train_linear_model(model_name=model_name, device=device, emb_size=2048)

In [ ]:
# torch.save(linear_model.state_dict(), f'../data/trained_models/{model_name}_cifar_linear_model0.905test_accuracy.pth')